In [56]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [57]:
tr = pd.read_csv('input/train.csv')

In [58]:
te = pd.read_csv('input/test.csv')
te['TripType'] = 0

In [59]:
d = pd.concat([tr, te])
d.TripType = d.TripType.astype(int)
del d['FinelineNumber']
del d['Upc']
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300700 entries, 0 to 653645
Data columns (total 5 columns):
DepartmentDescription    1298011 non-null object
ScanCount                1300700 non-null int64
TripType                 1300700 non-null int64
VisitNumber              1300700 non-null int64
Weekday                  1300700 non-null object
dtypes: int64(3), object(2)
memory usage: 59.5+ MB


In [60]:
e = pd.get_dummies(d, columns=['DepartmentDescription', 'Weekday', 'TripType'], dummy_na=True)
e['Weekday'] = d.Weekday
e['TripType'] = d.TripType
#e.info()

In [61]:
def sanitize(s):
    ret = s.replace(',', '').replace('&', 'AND').replace('/', ' ').replace(' ', '_')
    ret = ret.split('.')[0]
    return ret
e.columns = [sanitize(c) for c in e.columns]

In [ ]:
for c in e.columns:
    if c.startswith('DepartmentDescription_'):
        e[c] = e.ScanCount * e[c]

In [ ]:
def agg_visit(g):
    scp = int(g[g.ScanCount > 0].ScanCount.sum())
    scn = -int(g[g.ScanCount < 0].ScanCount.sum())
    sc = scp + scn
    ret = DataFrame({
            'ScanCountPos': [scp],
            'ScanCountNeg': [scn],
            'ScanCountPosRatio': [1.0*scp/sc],
            'ScanCountNegRatio': [1.0*scn/sc],
            'TripType': [int(g.TripType.min())],
            'Weekday': [g.Weekday.min()]
        })
    for c in g.columns:
        if c.startswith('DepartmentDescription_'):
            ret[c] = g[c].sum() / sc
    for c in g.columns:
        if c.startswith('Weekday_'):
            ret[c] = [int(g[c].min())]
    for c in g.columns:
        if c.startswith('TripType_'):
            ret[c] = [int(g[c].min())]
    return ret
g = e.groupby(by=['VisitNumber'])
h = g.apply(agg_visit)
h['VisitNumber'] = h.index.get_level_values(0)

In [ ]:
#h[:10].T

In [ ]:
h.to_csv('tmp/engineered_features_01.csv', index=False)